In [1]:
import requests

In [2]:
ENKI_API_SERVICE_ID = "enki"

In [3]:
KONG_HOST_IP = "localhost"
KONG_PORT = 8001

KEYCLOAK_URL = "http://localhost:8080/auth/"
USERNAME = "admin"
PASSWORD = 'Pa55w0rd'
MASTER_REALM_NAME = "master"

CLIENT_ID = "kong_test_client-4"


In [4]:
#def clean():
response = requests.get(f'http://{KONG_HOST_IP}:{KONG_PORT}/routes')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'http://{KONG_HOST_IP}:{KONG_PORT}/routes/{_id}')
response = requests.get(f'http://{KONG_HOST_IP}:{KONG_PORT}/services')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/{_id}')

ConnectionError: HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded with url: /routes (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fde0d391880>: Failed to establish a new connection: [Errno 61] Connection refused'))

# Create Kong client 

In [84]:
import uuid

'1bfafcce-6ada-4230-ac57-e7b9f15fcf8a'

In [68]:
from keycloak import KeycloakAdmin



#%%

keycloak_admin = KeycloakAdmin(server_url=KEYCLOAK_URL,
                               username=USERNAME,
                           password=PASSWORD,
                               verify=True)

In [70]:
keycloak_admin.create_client({
 "id":"4",
 "clientId":CLIENT_ID,
 "name":CLIENT_ID,
 "enabled": True,
 "redirectUris":[ "/front/*", "/api/*" ],
})

b''

In [71]:
client_secret_key = keycloak_admin.get_client_secrets("4")["value"]
client_secret_key

'9d0cd4b1-aa0b-4020-804a-c34af78f499e'

In [82]:
services = [
    {
        'name': "front_service",
        'url': f'http://nginx:7777/front',
        'path': "front"

    },
        {
        'name': "api_service",
        'url': f'http://nginx:7777/api',
        'path': "api"

    }
]

REALM_NAME = "master"
KEYCLOAK_HOST_IP="keycloak"
KEYCLOAK_PORT=8080

introspection_url = f'http://{KEYCLOAK_HOST_IP}:{KEYCLOAK_PORT}/auth/realms/{REALM_NAME}/protocol/openid-connect/token/introspect'
discovery_url = f'http://{KEYCLOAK_HOST_IP}:{KEYCLOAK_PORT}/auth/realms/{REALM_NAME}/.well-known/openid-configuration'
"http://localhost:8080/auth/realms/master/protocol/openid-connect/token/introspect"
print(introspection_url)
print(discovery_url)

http://keycloak:8080/auth/realms/master/protocol/openid-connect/token/introspect
http://keycloak:8080/auth/realms/master/.well-known/openid-configuration


In [83]:
for service in services:
    data = service
    
    # Create Service
    response = requests.post(f'http://{KONG_HOST_IP}:{KONG_PORT}/services', data=data)
    created_service_id = response.json()["id"]
    
    # Create route
    data = {
        'service.id': f'{created_service_id}',
        'paths[]': f'/{service["path"]}',
    }

    response = requests.post(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/{service["name"]}/routes', data=data)      
    # Configure OIDC 
    data = {
        'name': 'oidc',
        'config.client_id': f'{CLIENT_ID}',
        'config.client_secret': f'{client_secret_key}',
        'config.realm': f'{REALM_NAME}',
        'config.bearer_only': 'true',
        'config.introspection_endpoint':introspection_url,
        'config.discovery':discovery_url
    }

    response = requests.post(f'http://{KONG_HOST_IP}:{KONG_PORT}/services/{created_service_id}/plugins', data=data)

{'id': 'b1acb341-d3a0-4041-8052-fc28f6e0e632', 'tags': None, 'paths': ['/front'], 'destinations': None, 'headers': None, 'protocols': ['http', 'https'], 'strip_path': True, 'created_at': 1617960987, 'request_buffering': True, 'hosts': None, 'name': None, 'updated_at': 1617960987, 'snis': None, 'preserve_host': False, 'regex_priority': 0, 'methods': None, 'sources': None, 'response_buffering': True, 'https_redirect_status_code': 426, 'path_handling': 'v0', 'service': {'id': 'a5658547-7eff-4c8e-a5b9-062b1dc65bae'}}
{'created_at': 1617960987, 'id': '6cbc123a-6a1a-425d-9885-176df91f6330', 'tags': None, 'enabled': True, 'protocols': ['grpc', 'grpcs', 'http', 'https'], 'name': 'oidc', 'consumer': None, 'service': {'id': 'a5658547-7eff-4c8e-a5b9-062b1dc65bae'}, 'route': None, 'config': {'response_type': 'code', 'introspection_endpoint': 'http://keycloak:8080/auth/realms/master/protocol/openid-connect/token/introspect', 'filters': None, 'bearer_only': 'true', 'ssl_verify': 'no', 'session_secre

# Configure OIDC connect

In [76]:
f"http://{KONG_HOST_IP}:{KONG_PORT}/services/{created_service_id}/plugins"

'http://localhost:8001/services/d2b6908c-3fcd-4090-a3b5-9f0ae573196f/plugins'